In [ ]:
import sys
base_path = 'insert absolute path/RSE-pytorch-scaled'
sys.path.insert(0, base_path)
from transcribe.tools import predictAudio,load_target,thresholdingOnsets
import random
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
import os
import os.path

In [2]:
root_path = '../../maestro-v3.0.0'
dataset = pd.read_csv(os.path.join(root_path, 'maestro-v3.0.0.csv')).drop(["canonical_composer","canonical_title","year"],axis=1)
midis = dataset[dataset['split']=='test']['midi_filename'].to_list()

In [3]:
random.seed(a=1)
test_midis = random.choices(midis,k=20)

In [4]:
def extractOnsets(notes,scale=3):
    r = scale-(notes.shape[0]%scale) #scale=3 means a tolerance of 45ms with stride=165
    padded_notes = np.pad(notes, ((0, r), (0, 0)), mode='constant', constant_values=0)
    scaled_notes = (np.mean(padded_notes.reshape(padded_notes.shape[0]//scale, scale, 128), axis=1) > 0.5).astype(int)
    return (scaled_notes-np.pad(scaled_notes, ((0, 1), (0, 0)), mode='constant', constant_values=0)[1:,:])==-1

In [5]:
scores = []
for midi in test_midis:
    prediction = predictAudio(os.path.join(root_path, midi[:-4]+'wav'),16384)
    target = load_target(prediction.shape,os.path.join(root_path, midi))
    target_onsets = extractOnsets(target)
    target_onsets, pred_onsets,dual_threshold = thresholdingOnsets(prediction,target_onsets,0.5, 0.2, 0.005, 0.35, 4)
    macro = f1_score(target_onsets,pred_onsets,average='macro',zero_division=1)
    print(macro)
    scores.append(macro)

Processing...: 102it [01:34,  1.07it/s]                         


0.9928668478260869


Processing...: 449it [07:17,  1.03it/s]                         


0.8620072898968223


Processing...: 369it [06:07,  1.00it/s]                         


...

0.9387583570107804


Processing...: 173it [02:53,  1.00s/it]                         


0.9615025481314635


Processing...: 543it [09:04,  1.00s/it]                         


0.9478531318146854


In [6]:
np.mean(scores)

0.9200464291215406